# Supply Chain Process Compliance & EDA

This notebook analyzes:
- Planning process adherence (happy path analysis)
- Supplier lead time compliance
- Inventory diagnostics and exposure
- Planning parameter outlier detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append('..')
from utils import *

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load datasets
orders = pd.read_csv('../data/orders.csv', parse_dates=['order_date', 'planned_delivery', 'delivery_date'])
inventory = pd.read_csv('../data/inventory.csv')
products = pd.read_csv('../data/products.csv')
suppliers = pd.read_csv('../data/suppliers.csv')

print(f"📊 Dataset Overview:")
print(f"   Orders: {len(orders):,} records")
print(f"   Inventory: {len(inventory):,} items")
print(f"   Products: {len(products):,} items")
print(f"   Suppliers: {len(suppliers):,} suppliers")
print(f"   Date range: {orders['order_date'].min()} to {orders['order_date'].max()}")

## 1. Happy Path Analysis

In [ ]:
# Define happy path criteria
orders['on_time'] = orders['delivery_date'] <= orders['planned_delivery']
orders['quality_ok'] = orders['defect_rate'] <= 2.0
orders['process_compliant'] = (orders['mrp_compliance'] == 'Compliant') & (orders['setup_compliance'] == 'Compliant')
orders['happy_path'] = orders['on_time'] & orders['quality_ok'] & orders['process_compliant']

# Calculate happy path percentage
happy_path_pct = orders['happy_path'].mean() * 100
print(f"🎯 Happy Path Performance: {happy_path_pct:.1f}%")

# Breakdown by criteria
criteria_breakdown = {
    'On-Time Delivery': orders['on_time'].mean() * 100,
    'Quality OK': orders['quality_ok'].mean() * 100,
    'Process Compliant': orders['process_compliant'].mean() * 100,
    'Happy Path (All)': happy_path_pct
}

print("\n📋 Criteria Breakdown:")
for criteria, pct in criteria_breakdown.items():
    print(f"   {criteria}: {pct:.1f}%")

In [ ]:
# Visualize happy path performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Happy path by category
happy_by_cat = orders.groupby('category')['happy_path'].mean() * 100
happy_by_cat.plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Happy Path % by Category')
axes[0,0].set_ylabel('Happy Path %')
axes[0,0].tick_params(axis='x', rotation=45)

# Happy path by ABC class
happy_by_abc = orders.groupby('abc_class')['happy_path'].mean() * 100
happy_by_abc.plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('Happy Path % by ABC Class')
axes[0,1].set_ylabel('Happy Path %')

# Monthly trend
monthly_happy = orders.groupby(orders['order_date'].dt.to_period('M'))['happy_path'].mean() * 100
monthly_happy.plot(ax=axes[1,0], color='green')
axes[1,0].set_title('Happy Path Trend Over Time')
axes[1,0].set_ylabel('Happy Path %')
axes[1,0].axhline(y=85, color='red', linestyle='--', label='Target: 85%')
axes[1,0].legend()

# Failure mode analysis
failure_modes = {
    'Late Delivery': (~orders['on_time']).sum(),
    'Quality Issues': (~orders['quality_ok']).sum(),
    'Process Non-Compliance': (~orders['process_compliant']).sum()
}
axes[1,1].pie(failure_modes.values(), labels=failure_modes.keys(), autopct='%1.1f%%')
axes[1,1].set_title('Failure Mode Distribution')

plt.tight_layout()
plt.show()

## 2. Supplier Lead Time Compliance

In [ ]:
# Merge with supplier data
orders_with_suppliers = orders.merge(suppliers, on='supplier_id')

# Calculate lead time compliance
orders_with_suppliers['lead_time_compliance'] = orders_with_suppliers['lead_time'] <= orders_with_suppliers['lead_time_target']
orders_with_suppliers['lead_time_variance'] = orders_with_suppliers['lead_time'] - orders_with_suppliers['lead_time_target']

# Supplier performance summary
supplier_performance = orders_with_suppliers.groupby(['supplier_id', 'supplier_name']).agg({
    'lead_time_compliance': 'mean',
    'lead_time_variance': 'mean',
    'defect_rate': 'mean',
    'total_value': 'sum',
    'order_id': 'count'
}).round(2)

supplier_performance.columns = ['compliance_rate', 'avg_variance', 'avg_defect_rate', 'total_spend', 'order_count']
supplier_performance['compliance_rate'] *= 100

print("🚚 Top 10 Suppliers by Performance:")
print(supplier_performance.sort_values('compliance_rate', ascending=False).head(10))

In [ ]:
# Supplier performance visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Lead time compliance distribution
axes[0,0].hist(supplier_performance['compliance_rate'], bins=20, alpha=0.7, color='skyblue')
axes[0,0].set_title('Lead Time Compliance Distribution')
axes[0,0].set_xlabel('Compliance Rate (%)')
axes[0,0].axvline(x=95, color='red', linestyle='--', label='Target: 95%')
axes[0,0].legend()

# Lead time variance by supplier
top_suppliers = supplier_performance.nlargest(10, 'total_spend')
top_suppliers['avg_variance'].plot(kind='bar', ax=axes[0,1], color='orange')
axes[0,1].set_title('Lead Time Variance - Top 10 Suppliers')
axes[0,1].set_ylabel('Avg Variance (days)')
axes[0,1].tick_params(axis='x', rotation=45)

# Supplier performance matrix
axes[1,0].scatter(supplier_performance['compliance_rate'], supplier_performance['avg_defect_rate'], 
                 s=supplier_performance['total_spend']/1000, alpha=0.6)
axes[1,0].set_xlabel('Lead Time Compliance (%)')
axes[1,0].set_ylabel('Avg Defect Rate (%)')
axes[1,0].set_title('Supplier Performance Matrix')

# Country performance
country_perf = orders_with_suppliers.groupby('country')['lead_time_compliance'].mean() * 100
country_perf.plot(kind='bar', ax=axes[1,1], color='lightgreen')
axes[1,1].set_title('Lead Time Compliance by Country')
axes[1,1].set_ylabel('Compliance Rate (%)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Inventory Diagnostics

In [ ]:
# Inventory exposure analysis
inventory['stock_coverage'] = inventory['current_stock'] / inventory['avg_demand']
inventory['excess_stock'] = inventory['current_stock'] - inventory['eoq']
inventory['shortage_risk'] = inventory['rop'] - inventory['current_stock']
inventory['shortage_risk'] = inventory['shortage_risk'].clip(lower=0)

# Categorize inventory exposure
def categorize_exposure(row):
    if row['shortage_risk'] > 0:
        return 'Shortage Risk'
    elif row['excess_stock'] > row['eoq'] * 0.5:
        return 'Excess Stock'
    else:
        return 'Normal'

inventory['exposure_category'] = inventory.apply(categorize_exposure, axis=1)

print("📦 Inventory Exposure Summary:")
exposure_summary = inventory['exposure_category'].value_counts()
for category, count in exposure_summary.items():
    pct = count / len(inventory) * 100
    print(f"   {category}: {count} items ({pct:.1f}%)")

# Calculate financial impact
inventory_with_products = inventory.merge(products, on='product_id')
excess_value = (inventory_with_products['excess_stock'] * inventory_with_products['unit_cost']).sum()
shortage_items = len(inventory[inventory['exposure_category'] == 'Shortage Risk'])

print(f"\n💰 Financial Impact:")
print(f"   Excess inventory value: ${excess_value:,.0f}")
print(f"   Items at shortage risk: {shortage_items}")

In [ ]:
# Inventory diagnostics visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Stock coverage distribution
axes[0,0].hist(inventory['stock_coverage'], bins=30, alpha=0.7, color='skyblue')
axes[0,0].set_title('Stock Coverage Distribution (Days)')
axes[0,0].set_xlabel('Days of Coverage')
axes[0,0].axvline(x=30, color='red', linestyle='--', label='Target: 30 days')
axes[0,0].legend()

# Exposure category distribution
exposure_summary.plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
axes[0,1].set_title('Inventory Exposure Distribution')

# EOQ vs Current Stock
axes[1,0].scatter(inventory['eoq'], inventory['current_stock'], 
                 c=inventory['exposure_category'].map({'Normal': 'green', 'Excess Stock': 'red', 'Shortage Risk': 'orange'}),
                 alpha=0.6)
axes[1,0].plot([0, inventory['eoq'].max()], [0, inventory['eoq'].max()], 'k--', alpha=0.5)
axes[1,0].set_xlabel('EOQ')
axes[1,0].set_ylabel('Current Stock')
axes[1,0].set_title('EOQ vs Current Stock')

# Stock status by category
inventory_with_products = inventory.merge(products, on='product_id')
status_by_cat = pd.crosstab(inventory_with_products['category'], inventory_with_products['stock_status'])
status_by_cat.plot(kind='bar', stacked=True, ax=axes[1,1])
axes[1,1].set_title('Stock Status by Product Category')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Planning Parameter Outlier Detection

In [ ]:
# Detect outliers in planning parameters
def detect_outliers(df, column, method='iqr'):
    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return (df[column] < lower_bound) | (df[column] > upper_bound)
    elif method == 'zscore':
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        return z_scores > 3

# Detect outliers in key parameters
inventory['eoq_outlier'] = detect_outliers(inventory, 'eoq')
inventory['safety_stock_outlier'] = detect_outliers(inventory, 'safety_stock')
inventory['rop_outlier'] = detect_outliers(inventory, 'rop')

# Summary of outliers
outlier_summary = {
    'EOQ Outliers': inventory['eoq_outlier'].sum(),
    'Safety Stock Outliers': inventory['safety_stock_outlier'].sum(),
    'ROP Outliers': inventory['rop_outlier'].sum()
}

print("🔍 Planning Parameter Outliers:")
for param, count in outlier_summary.items():
    pct = count / len(inventory) * 100
    print(f"   {param}: {count} items ({pct:.1f}%)")

# Identify products with multiple outliers
inventory['total_outliers'] = (inventory['eoq_outlier'] + 
                              inventory['safety_stock_outlier'] + 
                              inventory['rop_outlier'])

high_outlier_products = inventory[inventory['total_outliers'] >= 2]
print(f"\n⚠️  Products with multiple parameter outliers: {len(high_outlier_products)}")

if len(high_outlier_products) > 0:
    print("\nTop products requiring parameter review:")
    print(high_outlier_products[['product_id', 'eoq', 'safety_stock', 'rop', 'total_outliers']].head())

In [ ]:
# Visualize parameter outliers
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# EOQ distribution with outliers
axes[0,0].hist(inventory['eoq'], bins=30, alpha=0.7, color='skyblue', label='Normal')
if inventory['eoq_outlier'].any():
    axes[0,0].hist(inventory[inventory['eoq_outlier']]['eoq'], bins=30, alpha=0.7, color='red', label='Outliers')
axes[0,0].set_title('EOQ Distribution')
axes[0,0].set_xlabel('EOQ')
axes[0,0].legend()

# Safety stock distribution with outliers
axes[0,1].hist(inventory['safety_stock'], bins=30, alpha=0.7, color='lightgreen', label='Normal')
if inventory['safety_stock_outlier'].any():
    axes[0,1].hist(inventory[inventory['safety_stock_outlier']]['safety_stock'], bins=30, alpha=0.7, color='red', label='Outliers')
axes[0,1].set_title('Safety Stock Distribution')
axes[0,1].set_xlabel('Safety Stock')
axes[0,1].legend()

# Parameter correlation matrix
param_corr = inventory[['eoq', 'safety_stock', 'rop', 'avg_demand']].corr()
sns.heatmap(param_corr, annot=True, cmap='coolwarm', center=0, ax=axes[1,0])
axes[1,0].set_title('Parameter Correlation Matrix')

# Outlier count by category
inventory_with_cat = inventory.merge(products[['product_id', 'category']], on='product_id')
outlier_by_cat = inventory_with_cat.groupby('category')['total_outliers'].sum()
outlier_by_cat.plot(kind='bar', ax=axes[1,1], color='orange')
axes[1,1].set_title('Total Outliers by Category')
axes[1,1].set_ylabel('Number of Outliers')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Key Insights Summary

In [ ]:
print("=== SUPPLY CHAIN PROCESS COMPLIANCE ANALYSIS ===")
print(f"\n🎯 OVERALL PERFORMANCE:")
print(f"   Happy Path Achievement: {happy_path_pct:.1f}%")
print(f"   On-Time Delivery: {orders['on_time'].mean()*100:.1f}%")
print(f"   Process Compliance: {orders['process_compliant'].mean()*100:.1f}%")
print(f"   Quality Performance: {orders['quality_ok'].mean()*100:.1f}%")

print(f"\n🚚 SUPPLIER PERFORMANCE:")
avg_compliance = supplier_performance['compliance_rate'].mean()
print(f"   Average Lead Time Compliance: {avg_compliance:.1f}%")
print(f"   Suppliers Above 95% Compliance: {(supplier_performance['compliance_rate'] >= 95).sum()}")
print(f"   Suppliers Below 80% Compliance: {(supplier_performance['compliance_rate'] < 80).sum()}")

print(f"\n📦 INVENTORY EXPOSURE:")
print(f"   Items at Shortage Risk: {(inventory['exposure_category'] == 'Shortage Risk').sum()}")
print(f"   Items with Excess Stock: {(inventory['exposure_category'] == 'Excess Stock').sum()}")
print(f"   Excess Inventory Value: ${excess_value:,.0f}")

print(f"\n⚙️  PLANNING PARAMETERS:")
total_outliers = sum(outlier_summary.values())
print(f"   Total Parameter Outliers: {total_outliers}")
print(f"   Products Needing Review: {len(high_outlier_products)}")

print(f"\n💡 TOP RECOMMENDATIONS:")
if happy_path_pct < 85:
    print(f"   🔴 CRITICAL: Happy path below target (85%). Focus on process standardization.")
if avg_compliance < 90:
    print(f"   🟡 HIGH: Supplier lead time compliance needs improvement.")
if excess_value > 100000:
    print(f"   🟡 MEDIUM: Significant excess inventory detected. Review EOQ parameters.")
if total_outliers > len(inventory) * 0.1:
    print(f"   🟡 MEDIUM: High number of planning parameter outliers. Standardization needed.")